## 🎯 Contexto del Notebook

### ¿Qué?
Optimización de punto de reorden (s) y cantidad de pedido (Q) minimizando costos de mantener inventario y ordenar.

### ¿Por qué?
Políticas manuales pueden generar exceso o faltantes. La optimización balancea costos y servicio.

### ¿Para qué?
- Reducir costos totales de inventario
- Mejorar nivel de servicio (reducir stockouts)
- Automatizar decisiones de reposición

### ¿Cuándo?
Ejecutar mensualmente o cuando cambian costos/demanda significativamente.

### ¿Cómo?
1. Calcular demanda promedio y variabilidad
2. Definir costos de orden y holding
3. Resolver modelo de optimización EOQ + safety stock

In [ ]:
import pandas as pd
import numpy as np
from src.utils.paths import DATA_RAW

In [ ]:
orders = pd.read_csv(DATA_RAW / 'orders.csv')
sku_demand = orders.groupby('sku').agg({'qty': ['mean', 'std', 'count']}).reset_index()
sku_demand.columns = ['sku', 'avg_demand', 'std_demand', 'periods']
print(sku_demand.head())

In [ ]:
# EOQ simplificado
holding_cost = 2.0  # $ por unidad-año
order_cost = 50.0   # $ por orden
sku_demand['EOQ'] = np.sqrt((2 * order_cost * sku_demand['avg_demand'] * sku_demand['periods']) / holding_cost)
sku_demand['safety_stock'] = 1.65 * sku_demand['std_demand']  # 95%
sku_demand['reorder_point'] = sku_demand['avg_demand'] + sku_demand['safety_stock']
print(sku_demand[['sku','EOQ','safety_stock','reorder_point']].head(10))